# Overview

Our project revolves around PDDLs. In this document, we demonstrate how we represent PDDLs and our implemented solution

## State
A state is represented by a Python dictionary. The key should be a string representing the fluent, and its corresponding value should be an Integer – $1$ or $0$ representing **True** or **False**.

### Example
Below is an example of a state of a person who is not hungry.

In [1]:
state = {'person': 1, 'hungry': 0}

## Action class
The action class represents an action in a PDDL. An action object takes four arguments
1. Preconditions: the preconditions for the action in the form of a dictionary
2. Effects: the applied effects of the action
3. Name: the name of the action
4. Cost: the cost for taking the action (used for A* algorithm) 

In [2]:
class Action:
    pre = {} #preconditions
    eff={} #effects
    cost=0
    name = ""
    def __init__(self, p, e, c, n):
        self.pre = p
        self.eff = e
        self.name = n
        self.cost = c
        
    def formatFluents(self, d):
        string = ""
        for k in d.keys():
            if(d[k]==0):
                string += "\n- " + k
            else:
                string += "\n+ " + k
        return string

    def __str__(self):
        return f'Name: {self.name} \nCost: {self.cost} \nPreconditions: {self.formatFluents(self.pre)} \nEffects: {self.formatFluents(self.eff)}'

### Example
Below is an example of an action being created

In [3]:
a = Action(p={"person":1, "hungry":0, "hasCake":1},e={"hasCake":0, "person":0},c=2,n="EatCake B")

## Core functions

### Below are core functions used for the Forward Search (A*) algorithm

In [4]:
# Get only positive fluents
def getPositiveFluents(state):
    return {key:val for key, val in state.items() if val == 1}


# Get only negative fluents
def getNegativeFluents(state):
    return {key:val for key, val in state.items() if val == 0}


# Check if two states match (match positive fluents only)
def statesMatch(s1,s2):
    s1_pos = getPositiveFluents(s1).copy()
    s2_pos = getPositiveFluents(s2).copy()
    return s1_pos == s2_pos


# Checks if state is a subset of the target – logic rule for checking goal state and applicable actions
def stateSubset(state, target):
    positives = getPositiveFluents(state).copy()
    targetPos = getPositiveFluents(target).copy()
    targetNeg = getNegativeFluents(target).copy()
    subsetPos = targetPos.items() <= positives.items()
    subsetNeg = any(p in positives for p in targetNeg) == False
    return subsetPos & subsetNeg


# Checks if a state fulfills the requirements for a goal state
def atGoal(state, goal):
    remainders = getPositiveFluents(state).copy().items() - getPositiveFluents(goal).copy().items()
    worldCompliment = remainders - getNegativeFluents(goal).copy().items() == set()
    return stateSubset(state, goal) & worldCompliment


# Checks if an action is applicable for a given state
def isApplicable(state, action):
    return stateSubset(state, action.pre)
    

# For a state and all possible actions, return the [applicable actions] for the given state
def applicableActions(state, actions):
    arr = []
    for a in actions:
        if isApplicable(state, a):
            arr.append(a)
    return arr


# Given a state and an action, applies the action and returns the following state if all preconditions are met
def applyAction(state, action):    
    if isApplicable(state, action) == False:
        raise Exception(f'Action {action.name} cannot be applied to state {str(state)}')
    newState = state.copy()
    effects = action.eff.copy()
    for k in effects.keys():
        newState[k] = effects[k]
        if effects[k] == 0:
            newState.pop(k)
    return newState

## A*

Below is our implementation of the A* algorithm. First, we design a custom Node class for the algorithm to use. Second, we define the heuristic that we use 

Finally, for A*, the function takes a start state, a goal state, and all actions for the PDDL as input. The output is the list of actions (the path) which the A* algorithm finds. 

In [5]:
# Node class for frontier
class Node:
    
    def __init__(self, state, cost, path):
        self.state = state
        self.cost = cost
        self.path = path

    def __str__(self):
        pathNames = [p.name for p in self.path]
        pathStr = '[' + ', '.join(pathNames) + ']'
        return f'State: {self.state}, Cost: {self.cost}, Path: {pathStr}'

In [9]:
### TODO: Place heuristic here
def heuristic(nextState, goal):
    return 0

In [10]:
def aStar(start, goal, actions):

    # Init start node, frontier and visited list
    init = Node(start, 0, [])
    frontier = [init]
    visited = [init]

    while True:
        
        # Get first node from frontier
        head = frontier.pop(0)

        state = head.state.copy()

        # When A* has found a path, return it
        if (atGoal(state, goal)):
            print(f'A* found a path with cost: {head.cost}')
            return head.path

        applicActions = applicableActions(state, actions)
        
        for action in applicActions:
            nextState = applyAction(state, action).copy()
            newPath = head.path + [action]
            cost = heuristic(nextState, goal) + head.cost + action.cost
            newNode = Node(nextState, cost, newPath)

            # Only apply action if we haven't visited the state before
            vis = False
            for i in range(len(visited)):
                if visited[i].state == nextState:
                    vis = True
                    if cost < visited[i].cost:
                        visited[i] = newNode
                        frontier.append(newNode)
            if vis == False:
                visited.append(newNode)
                frontier.append(newNode)

        # Sort frontier based on costs
        frontier = sorted(frontier, key=lambda x:x.cost)
        


### Example

Below is an example of the A* algorithm on the PDDL from the lecture on PDDLs

In [11]:
goal = {'person':1,'hungry':0, "hasCake":1, "eatenCake":1}
start = {'person':1}

a1 = Action(p={"person":1, "hungry":0, "hasCake":1},e={"hasCake":0, "person":0},c=1,n="EatCake B")
a2 = Action(p={"person":1, "hungry":1, "hasCake":1},e={"hasCake":0, "eatenCake":1,"hungry":0},c=1,n="EatCake A")
a3 = Action(p={"person":1,"hasMix":1},e={"hasCake":1,"hasMix":0},c=1,n="MakeCake")
a4 = Action(p={"person":1,"hungry":0},e={"hungry":1},c=1,n="Wait")
a5 = Action(p={"person":1},e={"hasMix":1},c=1,n="Go Shopping")

actions = [a1,a2,a3,a4,a5]

solution = aStar(start, goal, actions)
print(f'Solution: {[a.name for a in solution]}')

A* found a path with cost: 6
Solution: ['Wait', 'Go Shopping', 'MakeCake', 'EatCake A', 'Go Shopping', 'MakeCake']


## Automated tests
Below are automated tests for the functions and algorithms implemented.

In [14]:
def testStatesMatch():
    
    # Test null state
    s1 = {}
    s2 = {}
    assert(statesMatch(s1,s2))
    
    # Test equal state
    s1 = {'person':1,'hungry':0}
    s2 = {'person':1,'hungry':0}
    assert(statesMatch(s1,s2))
    
    # Test pass by omission
    s1 = {'person':1}
    s2 = {'person':1, 'hungry':0}
    assert(statesMatch(s1,s2))
    
    # Test non-equal fluents state
    s1 = {'person':1,'hungry':0}
    s2 = {'person':0,'hungry':1}
    assert(statesMatch(s1,s2) == False)
    
    # Test different states
    s1 = {'person':1}
    s2 = {'hungry':1}
    assert(statesMatch(s1,s2) == False)
    
    # Test different states
    s1 = {'person':1}
    s2 = {'hungry':0}
    assert(statesMatch(s1,s2) == False)
    
    # Test different states
    s1 = {'person':1}
    s2 = {'person':1, 'hungry':1}
    assert(statesMatch(s1,s2) == False)
    
    print(f'All test cases for function "statesMatch" passed')

    
def testGetPositiveFluents():
    
    s = {'person':0,'hungry':0}
    assert(getPositiveFluents(s) == {})
    
    s = {'person':1,'hungry':0}
    assert(getPositiveFluents(s) == {'person':1})
    
    s = {'person':1, 'hungry':1, 'angry': 1}
    assert(getPositiveFluents(s) == {'person':1, 'hungry':1, 'angry': 1})
    
    s = {'person':1}
    assert(getPositiveFluents(s) == {'person':1})
    
    print(f'All test cases for function "getPositiveFluents" passed')
    
    
def testApplicableActions():
    
    # Test cases from powerpoint examples
    start = {'person':1}
    applActions = applicableActions(start, actions)
    assert(all(a in [a4,a5] for a in applActions))
    assert(any(a in [a1,a2,a3] for a in applActions) == False)
    
    s1 = {'person': 1, 'hasMix': 1}
    applActions = applicableActions(s1, actions)
    assert(all(a in [a3,a4,a5] for a in applActions))
    assert(any(a in [a1,a2] for a in applActions) == False)
    
    s2 = {'person': 1, 'hungry': 1}
    applActions = applicableActions(s2, actions)
    assert(all(a in [a5] for a in applActions))
    assert(any(a in [a1,a2,a3,a4] for a in applActions) == False)
    
    s3 = {'person': 1, 'hasCake': 1}
    applActions = applicableActions(s3, actions)
    assert(all(a in [a1,a4,a5] for a in applActions))
    assert(any(a in [a2,a3] for a in applActions) == False)
    
    s4 = {'person': 1, 'hungry': 1, 'hasMix': 1}
    applActions = applicableActions(s4, actions)
    assert(all(a in [a3,a5] for a in applActions))
    assert(any(a in [a1,a2,a4] for a in applActions) == False)
    
    print(f'All test cases for function "applicableActions" passed')

    
def testApplyAction():
    
    # Test added fluents
    s = {'a':1}
    a = Action(p={'a':1},e={'b':1},c=0,n="")
    assert(applyAction(s, a) == {'a':1, 'b':1})
    a = Action(p={'a':1},e={'b':1, 'c':1},c=0,n="")
    assert(applyAction(s, a) == {'a':1, 'b':1, 'c':1})
    
    # Test omission
    s = {'a':1}
    a = Action(p={'a':1, 'c':0},e={'b':1, 'c':1},c=0,n="")
    assert(applyAction(s, a) == {'a':1, 'b':1, 'c':1})
    
    # Test negative (applyAction should throw exception)
    try:
        s = {'a':1, 'c':1}
        a = Action(p={'a':1, 'c':0},e={'b':1, 'c':1},c=0,n="")
        raise Exception(f'Test negative failed')
    except:
        pass
    
    print(f'All test cases for function "applyAction" passed')


# Tests valid solutions to the cake-PDDL (See slide 63 - PDDL lecture)
def testPDDLsolutions():
    
    # Test valid path from start to goal
    s1 = applyAction(start, a5)
    s3 = applyAction(s1, a3)
    s5 = applyAction(s3, a5)
    s9 = applyAction(s5, a4)
    s11 = applyAction(s9, a2)
    g = applyAction(s11, a3)
    assert(atGoal(g, goal))
    
    # Test another valid path from start to goal
    s2 = applyAction(start, a4)
    s4 = applyAction(s2, a5)
    s7 = applyAction(s4, a3)
    s10 = applyAction(s7, a2)
    s11_v2 = applyAction(s10, a5)
    
    # sanity check - check that s11 from both versions are identical
    assert(statesMatch(s11, s11_v2)) 
    
    g2 = applyAction(s11_v2, a3)
    assert(atGoal(g2, goal))
    
    print(f'All test cases for function "testPDDLsolutions" passed')

    
# Validate a PDDL solution
def validatePDDL(start, goal, solution):
    s = start
    for a in solution:
        s = applyAction(s, a)
    assert(atGoal(s, goal))
    print(f'The given PDDL solution is valid')


def testAtGoal():
    s = {'atGoal':1}
    goal = {'atGoal': 1}
    assert(atGoal(s,goal))
    
    s = {'atGoal':1, 'happy':0}
    goal = {'atGoal': 1}
    assert(atGoal(s,goal))
    
    s = {'atGoal':0}
    goal = {'atGoal': 1}
    assert(atGoal(s,goal) == False)
    
    s = {'notAtGoal':1}
    goal = {'atGoal': 1}
    assert(atGoal(s,goal) == False)
    
    print(f'All test cases for function "atGoal" passed')


In [16]:
testStatesMatch()
testGetPositiveFluents()
testApplicableActions()
testApplyAction()
testPDDLsolutions()
testAtGoal()

All test cases for function "statesMatch" passed
All test cases for function "getPositiveFluents" passed
All test cases for function "applicableActions" passed
All test cases for function "applyAction" passed
All test cases for function "testPDDLsolutions" passed
All test cases for function "atGoal" passed
